# Advent of Code 2023

[Website](https://adventofcode.com/2023)

## Day 1: Trebuchet?!

What is the sum of all of the calibration values?

In [ ]:
import re

def get_digits(line):
      return re.findall(r'\d', line)

def get_calibration_value(line):
    digits = get_digits(line)
    return int(digits[0] + digits[-1]) if len(digits) else 0

def get_calibration_sum(lines):
    return sum([get_calibration_value(line) for line in lines])

lines = open('01_input.txt', 'r').readlines()
get_calibration_sum(lines)

What is the sum of all of the calibration values?

In [ ]:
def get_digits(line):
    numbers = {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', \
               'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9'}

    digits = []
    for start_pos in range(len(line)):
        digit = next((numbers[number] for number in numbers if line[start_pos:].startswith(number)), None)
        if digit != None:
            digits.append(digit)
    return digits

get_calibration_sum(lines)

## Day 2: Cube Conundrum

What is the sum of the IDs of those games?

In [ ]:
def get_games(lines):
    games = []
    for game_str in [line.strip().split(':')[1].split(';') for line in lines]:
        subsets =[]
        for subset_str in game_str:
            subset = { 'blue': 0, 'green': 0, 'red': 0}
            for cube_str in subset_str.split(','):
                cubes = cube_str.split()
                subset[cubes[1]] = int(cubes[0])
            subsets.append(subset)
        games.append(subsets)
    return games

def is_possible(subset):
    max_num_cubes = {'blue': 14, 'green': 13, 'red': 12}
    return all(subset[color] <= max_num_cubes[color] for color in max_num_cubes.keys())

def get_possible_game_ids(games):
    possible_game_ids = []
    for game_id in range(len(games)):
        if all(is_possible(subset) for subset in games[game_id]):
            possible_game_ids.append(game_id+1)
    return possible_game_ids

lines = open('02_input.txt', 'r').readlines()

games = get_games(lines)
sum(get_possible_game_ids(games))

What is the sum of the power of these sets?

In [ ]:
from math import prod

def get_powers(games):
    powers = []
    for game in games:
        min_cube_num = {'blue': 0, 'green': 0, 'red': 0}
        for subset in game:
            for color in subset:
                min_cube_num[color] = max(min_cube_num[color], subset[color])
        powers.append(prod(min_cube_num.values()))
    return powers

sum(get_powers(games))

## Day 3: Gear Ratios

What is the sum of all of the part numbers in the engine schematic?

In [ ]:
from collections import namedtuple

def get_numbers(lines):
    Number = namedtuple('number', 'position number')
    numbers = []
    for row_id, line in enumerate(lines):
        column_id = 0
        while column_id < len(line):
            if not line[column_id].isdigit():
                column_id += 1
                continue
            position = (row_id, column_id)
            number = line[column_id]
            while column_id < len(line)-1 and line[column_id+1].isdigit():
                column_id += 1
                number += line[column_id]
            numbers.append(Number(position, int(number)))
            column_id += 1
    return numbers

def is_valid_position(position, matrix):
    return position[0] >= 0 and position[0] < len(matrix) and \
           position[1] >= 0 and position[1] < len(matrix[0])

def is_symbol(position, matrix):
    char = matrix[position[0]][position[1]]
    return not char.isdigit() and char != '.'

def has_symbol_as_neighbor(position, matrix):
    for row_move in [-1, 0, 1]:
        for column_move in [-1, 0, 1]:
            neighbor_position = (position[0] + row_move, position[1] + column_move)
            if is_valid_position(neighbor_position, matrix) and is_symbol(neighbor_position, matrix):
                return True
    return False

def get_part_number_infos(numbers, matrix):
    part_number_infos = []
    for number_info in numbers:
        row_id, column_id = number_info.position
        number = number_info.number
        if any(has_symbol_as_neighbor((row_id, column_id + column_move), matrix) for column_move in range(len(str(number)))):
            part_number_infos.append(number_info)
    return part_number_infos

lines = open('03_input.txt', 'r').readlines()
lines = [line.strip() for line in lines]

numbers = get_numbers(lines)
matrix = [[char for char in row] for row in lines]
part_number_infos = get_part_number_infos(numbers, matrix)
part_numbers = [info.number for info in part_number_infos]
sum(part_numbers)

What is the sum of all of the gear ratios in your engine schematic?

In [ ]:
from math import prod

def get_star_positions(lines):
    star_positions = []
    for row_id, line in enumerate(lines):
        for column_id, char in enumerate(line):
            if char == '*':
                star_positions.append((row_id, column_id))
    return star_positions

def are_neighbors(star_position, part_number_info):
    for row_move in [-1, 0, 1]:
        for column_move in [-1, 0, 1]:
            neighbor_position = (star_position[0] + row_move, star_position[1] + column_move)
            for number_colum_move in range(len(str(part_number_info.number))):
                if part_number_info.position[0] == neighbor_position[0] and part_number_info.position[1]+number_colum_move == neighbor_position[1]:
                    return True
    return False

def get_neighboring_part_numbers(star_position, part_number_infos):
    return [number_info.number for number_info in part_number_infos if are_neighbors(star_position, number_info)]

def get_gear_ratios(star_positions, part_number_infos):
    gear_ratios = []
    for star_position in star_positions:
        neighboring_part_numbers = get_neighboring_part_numbers(star_position, part_number_infos)
        if len(neighboring_part_numbers) == 2:
            gear_ratios.append(prod(neighboring_part_numbers))
    return gear_ratios

star_positions = get_star_positions(lines)
gear_ratios = get_gear_ratios(star_positions, part_number_infos)
sum(gear_ratios)